In [14]:
#install required libraries
!pip install openai pinecone-client sentence-transformers
!pip install --upgrade pinecone-client
!pip install sentence-transformers
!pip install nltk transformers

In [15]:
import os

from getpass import getpass
os.environ["PINECONE_API_KEY"] = getpass("Enter Pinecone API Key:")

Enter Pinecone API Key:··········


In [16]:
import os
from pinecone import Pinecone, ServerlessSpec

# Access Pinecone API key securely
pinecone_api_key = os.environ["PINECONE_API_KEY"]
pinecone = Pinecone(api_key=pinecone_api_key)

# Index name
index_name = 'rag-qa-bot0'

# Check if the index exists; if not, create it
if index_name not in [i.name for i in pinecone.list_indexes()]:
    pinecone.create_index(
        name=index_name,
        dimension=384,  # Matches embedding model dimension
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

# Connect to the index
index = pinecone.Index(index_name)


In [18]:
import json

# Update the path to the SQuAD dataset
file_path = '/content/train-v2.0.json'

with open(file_path, 'r') as file:
    squad_data = json.load(file)

In [19]:
documents = []

for article in squad_data['data']:
    for paragraph in article['paragraphs']:
        context = paragraph['context']
        qa_pairs = []
        for qa in paragraph['qas']:
            if not qa['is_impossible']:
                question = qa['question']
                answer = qa['answers'][0]['text']  # Use the first answer
                qa_pairs.append({"question": question, "answer": answer})
        documents.append({"text": context, "qa": qa_pairs})

print(f"Parsed {len(documents)} documents.")

# Generate Embeddings Locally
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')  # Use 384-dimensional embeddings

for doc in documents:
    doc["embedding"] = model.encode(doc["text"]).tolist()

print("Generated embeddings locally.")

# Save embeddings to a file
with open("squad_documents_with_embeddings.json", "w") as f:
    json.dump(documents, f)

print("Saved embeddings to 'squad_documents_with_embeddings.json'.")

Parsed 19035 documents.
Generated embeddings locally.
Saved embeddings to 'squad_documents_with_embeddings.json'.


In [20]:
import hashlib

for doc in documents:
    # Use a hash of the text as a unique ID
    doc["id"] = hashlib.md5(doc["text"].encode()).hexdigest()

    # Convert the 'qa' field to a JSON string
    metadata = {"qa": json.dumps(doc["qa"])}

    # Upsert data into Pinecone
    index.upsert([(doc["id"], doc["embedding"], metadata)])

print("Indexed documents in Pinecone.")

Indexed documents in Pinecone.


In [1]:
!pip install openai==0.28

In [4]:
import os

from getpass import getpass
os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API Key:")
os.environ["PINECONE_API_KEY"] = getpass("Enter Pinecone API Key:")

Enter OpenAI API Key:··········
Enter Pinecone API Key:··········


In [7]:
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec
import openai
import json
import time
import os

# Access Pinecone API key securely
pinecone_api_key = os.environ["PINECONE_API_KEY"]
pinecone = Pinecone(api_key=pinecone_api_key)

# Index name
index_name = 'rag-qa-bot0'

# Check if the index exists; if not, create it
if index_name not in [i.name for i in pinecone.list_indexes()]:
    pinecone.create_index(
        name=index_name,
        dimension=384,  # Matches embedding model dimension
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

# Connect to the index
index = pinecone.Index(index_name)

# Load the local embedding model
query_embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Define the function to generate embeddings locally
def get_local_embedding(text):
    return query_embedding_model.encode(text).tolist()

def retrieve_and_generate(query):
    # Generate query embedding locally
    query_embedding = get_local_embedding(query)

    # Retrieve relevant documents
    search_results = index.query(
        vector=query_embedding,
        top_k=1,  # Limit to top 1 for fewer tokens
        include_metadata=True
    )

    # Parse the retrieved Q&A metadata
    retrieved_texts = []
    for match in search_results['matches']:
        qa_data = json.loads(match['metadata']['qa'])  # Parse JSON string
        retrieved_texts.extend(qa_data)

    # Construct the context using retrieved Q&A pairs
    prompt_context = " ".join([
        f"Q: {qa['question']} A: {qa['answer']}"
        for qa in retrieved_texts
    ])

    # Set OpenAI API key
    openai_api_key = os.environ["OPENAI_API_KEY"]
    openai.api_key = openai_api_key

    # Retry logic for OpenAI API
    retries = 0
    max_retries = 5
    wait_time = 2  # Start with 2 seconds wait

    while retries < max_retries:
        try:
            # Use OpenAI API for response generation
            messages = [
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"Context: {prompt_context}\n\nQuery: {query}\n\nAnswer:"}
            ]
            response = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                messages=messages,
                max_tokens=100  # Limit token usage
            )
            return response['choices'][0]['message']['content'].strip()
        except openai.error.RateLimitError:
            retries += 1
            print(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
            wait_time *= 2  # Exponential backoff
        except Exception as e:
            print(f"Unexpected error: {e}")
            raise e

    raise RuntimeError("Max retries exceeded. Please check your OpenAI API limits.")

In [8]:
# Test the RAG bot
query = "what is the process of construction"
try:
    response = retrieve_and_generate(query)
    print(f"Response: {response}")
except RuntimeError as e:
    print(f"Error: {e}")

Response: The process of construction typically involves several key stages:

1. **Planning and Design**: This initial phase includes defining the project scope, conducting feasibility studies, and creating architectural designs. The plans outline the project specifications, budget, and timeline.

2. **Permitting and Approvals**: Before construction begins, necessary permits and approvals must be obtained from local authorities. This ensures that the project complies with zoning laws, building codes, and safety regulations.

3. **Site Preparation**: The construction site
